In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import random
import re
import time
from datetime import datetime, time as dt_time
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Final Project/consolidated_data.csv', header='infer', low_memory=False, parse_dates=['Date'])

# **Deduplication**

In [ ]:
data.shape

(6502752, 56)

In [ ]:
data.drop_duplicates(subset=data.columns[1:], inplace=True)

In [ ]:
data.shape

(6461513, 56)

In [ ]:
data.head()

,Id,Date,Sequence,MealName,DishName,Calcium_value,Calories_value,Carbs_value,Chol_value,Fat_value,...,Mon Fat_goal,Ply Fat_goal,Potass._goal,Protein_goal,Sat Fat_goal,Sodium_goal,Sugar_goal,Trn Fat_goal,Vit A_goal,Vit C_goal
0,1,2014-09-14,1,MY food,"my - McDonalds Espresso Pronto® Flat White, 2 ...",NaN,412,29,NaN,24.0,...,NaN,NaN,NaN,160.0,NaN,2300.0,119.0,NaN,NaN,NaN
1,1,2014-09-14,1,MY food,Quest Bar - Banana Nut Muffin Natural Protein ...,NaN,170,25,NaN,5.0,...,NaN,NaN,NaN,160.0,NaN,2300.0,119.0,NaN,NaN,NaN
2,1,2014-09-14,1,MY food,"Uncle Tobys Australia - Vita Brits, 3 Biscuits...",NaN,176,33,NaN,1.0,...,NaN,NaN,NaN,160.0,NaN,2300.0,119.0,NaN,NaN,NaN
3,1,2014-09-14,1,MY food,"Pauls - Smarter White Milk, 600 ml",NaN,342,34,NaN,12.0,...,NaN,NaN,NaN,160.0,NaN,2300.0,119.0,NaN,NaN,NaN
4,1,2014-09-14,1,MY food,"Quest Bar - Cookies and Cream, 1 bar",NaN,180,22,NaN,7.0,...,NaN,NaN,NaN,160.0,NaN,2300.0,119.0,NaN,NaN,NaN


# **Split data into train and test and save.**

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
train.to_csv('/content/drive/MyDrive/Final Project/train.csv', index=False)

In [ ]:
test.to_csv('/content/drive/MyDrive/Final Project/test.csv', index=False)

# **Read Train/Test**

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Final Project/train.csv', low_memory=False, parse_dates=['Date'])

In [ ]:
train.head()

,Id,Date,Sequence,MealName,DishName,Calcium_value,Calories_value,Carbs_value,Chol_value,Fat_value,...,Mon Fat_goal,Ply Fat_goal,Potass._goal,Protein_goal,Sat Fat_goal,Sodium_goal,Sugar_goal,Trn Fat_goal,Vit A_goal,Vit C_goal
0,975,2014-12-03,2,Lunch,Lays - Kettle Cooked 40% Less Fat Sea Salt and...,NaN,180,27,NaN,8,...,NaN,NaN,NaN,136.0,NaN,2500.0,73.0,NaN,NaN,NaN
1,7787,2015-03-15,5,Dinner,"Idaho potato - Goldust, 296 g",NaN,220,52,NaN,0,...,NaN,NaN,NaN,179.0,NaN,2500.0,55.0,NaN,NaN,NaN
2,5795,2014-12-09,3,Meal Two,"Mrvh Kroger - Green Onions, Raw, 0.5 oz (15g)",NaN,5,1,NaN,0,...,NaN,NaN,NaN,197.0,NaN,NaN,102.0,NaN,NaN,NaN
3,4186,2015-03-31,2,Meal 1,"Mayver's Original Super Spread - Nut Spread, 20 g",NaN,131,5,NaN,11,...,NaN,NaN,NaN,221.0,NaN,NaN,65.0,NaN,NaN,NaN
4,4897,2015-02-19,2,Lunch,"Kens - Light Honey Mustard Dressing, 0.5 tbsp",NaN,20,NaN,NaN,NaN,...,NaN,NaN,NaN,87.0,NaN,2300.0,65.0,NaN,NaN,NaN


# Divide mealtime columns into following parts:
     

1. 'breakfast',
      
2. 'lunch',

3. 'dinner',
     
4. 'snack',
     
5. 'drinks',
     
6. 'supplement',

7. 'workout',










In [ ]:
def process_MealName_column(meal_name):
  data = meal_name.copy()
  del meal_name
  data['MealName'] = data['MealName'].str.lower()

  data['MealName'] = data['MealName'].str.replace(r'[^a-zA-Z0-9\s:]', '', regex=True)
  data['MealName'] = data['MealName'].str.replace(r'th|rd|st|nd|#', '', regex=True)
  data['MealName'] = data['MealName'].str.replace(r'\s+', ' ', regex=True)


  breakfast_keywords = ['morni', 'first', 'break', 'wake up', 'meal 1', 'meal 2', 'meal one', 'meal two', '1 meal', '2 meal', 'one meal', 'two meal', 'one', 'two', 'early']
  lunch_keywords = ['afternoon', 'noon', 'lunch', 'mid', 'meal 3', 'meal 4', 'meal three', 'meal four', '3 meal', '4 meal', 'three meal', 'four meal', 'final', 'four', 'three']
  dinner_keywords = ['dinner','sleep', 'supper', 'evening', 'night', 'late', 'bedtime', 'before bed', 'meal 5', 'meal 6', 'meal five', 'meal six', '5 meal', '6 meal', 'five meal', 'six meal', 'five', 'six']
  snack_keywords = ['snack', 'desert', 'sweet']
  drinks_keywords = ['drink', 'tea', 'coffee', 'liquid', 'alcohol', 'beverage', 'fluid']
  supplement_keywords = ['supplement', 'vitamin', 'pill', 'protein', 'addition']
  workout_keywords = ['workout', 'work']


  meal_names = data['MealName'].values

  def convert_to_time_format(text):
      pattern = re.compile(r'(\d{1,2})(:?\d{2})?\s*(am|pm)?', flags=re.IGNORECASE)
      matches = pattern.findall(text.lower())

      if matches:
          times = []
          for match in matches:
              hour = int(match[0])
              minute = int(match[1][1:]) if match[1] else 0
              period = match[2].lower() if match[2] else None

              if period:
                  if 1 <= hour <= 12:
                      if period == 'pm' and hour < 12:
                          hour += 12
                      elif period == 'am' and hour == 12:
                          hour = 0
              else:
                  if hour == 24 and minute == 0:
                      hour = 0
                  elif hour >= 24 or minute >= 60:
                      continue

              times.append((hour, minute))

          if times:
              return times

      return None

  def date_in_range(meal_name, meal_type):
      times = convert_to_time_format(meal_name)

      if times:
          for hour, minute in times:
              if meal_type == 'breakfast' and (4 <= hour < 12 or (hour == 12 and minute == 0)):
                  return True
              elif meal_type == 'lunch' and (12 <= hour < 18 or (hour == 18 and minute == 0)):
                  return True
              elif meal_type == 'dinner' and (18 <= hour <= 23 or (hour == 0 and minute == 0)):
                  return True

              if meal_type == 'breakfast' and (hour in range(1, 13) or hour == 0) and 'am' in meal_name:
                  return True

      return False


  categorized_meal_names = []

  for meal_name in meal_names:
      lower_case_meal_name = meal_name.lower()

      if any(keyword in lower_case_meal_name for keyword in breakfast_keywords) or date_in_range(lower_case_meal_name, 'breakfast'):
          categorized_meal_names.append('breakfast')
          continue

      if any(keyword in lower_case_meal_name for keyword in lunch_keywords) or date_in_range(lower_case_meal_name, 'lunch'):
          categorized_meal_names.append('lunch')
          continue

      if any(keyword in lower_case_meal_name for keyword in dinner_keywords) or date_in_range(lower_case_meal_name, 'dinner'):
          categorized_meal_names.append('dinner')
          continue

      if any(keyword in lower_case_meal_name for keyword in snack_keywords):
          categorized_meal_names.append('snack')
          continue

      if any(keyword in lower_case_meal_name for keyword in drinks_keywords):
          categorized_meal_names.append('drinks')
          continue

      if any(keyword in lower_case_meal_name for keyword in supplement_keywords):
          categorized_meal_names.append('supplement')
          continue

      if any(keyword in lower_case_meal_name for keyword in workout_keywords):
          categorized_meal_names.append('workout')
          continue
      categorized_meal_names.append(meal_name)

  data['MealName'] = categorized_meal_names


  condition = [
    (data['MealName'] == 'breakfast'),
    (data['MealName'] == 'lunch'),
    (data['MealName'] == 'dinner'),
    (data['MealName'] == 'snack'),
    (data['MealName'] == 'drinks'),
    (data['MealName'] == 'supplement'),
    (data['MealName'] == 'workout'),

  ]
  choices = [
      'breakfast',
      'lunch',
      'dinner',
      'snack',
      'drinks',
      'supplement',
      'workout',
  ]

  data['MealName'] = np.select(condition, choices, default='unknown')
  return data

# **Convert all columns into numeric values**

In [ ]:
def convert_objects_into_numeric(data):
  colls = data.columns
  colls_to_check = colls[11:]
  for col in colls_to_check:
    if data[col].dtype == 'object':
      data[col] = pd.DataFrame(data[col]).apply(lambda x: x.str.replace(',', '.').str.replace(r'\.(?![^.]*$)', '', regex=True).astype(float))
  return data

# **Get minimum and maximum of each columns of training data and save to use same values for test data**

In [ ]:
def get_min_max(data):
  min_max_lst = []
  for i in data.columns:
    if i != 'DishName' and i != 'Id':
      min_max_lst.append([i, data[i].max(), data[i].min()])
  df = pd.DataFrame(min_max_lst, columns=['Column', 'Max', 'Min'])
  return df

# **Normalization of the data**

In [ ]:
def scale_data(data):
    min_max = pd.read_csv('/content/drive/MyDrive/Final Project/min_max.csv')
    #data.drop('Id', axis=1, inplace=True)
    cols_ = data.columns
    new_cols = cols_[:10].tolist() + cols_[11:].tolist() + [cols_[10]]
    data = data[new_cols]

    for i in data.columns[1:]:
        if i != 'DishName':
            min_val = min_max[min_max['Column'] == i]['Min'].values[0]
            max_val = min_max[min_max['Column'] == i]['Max'].values[0]
            if min_val == max_val:
                print(f"Column {i} has min_val == max_val == {min_val}. Filling this column with zeros.")
                data[i] = 0
            else:
                data[i] = (data[i] - min_val) / (max_val - min_val)

    return data


# **Get absolute values of indicated columns**

In [ ]:
def get_abs(data):
  for i in data.columns[11:-2]:
      data[i] = abs(data[i])
  return data

# **Apply all processes in "preprocess_data" function**

In [ ]:
def preprocess_data(data):
  data['Day'] = data['Date'].dt.day
  data['Month'] = data['Date'].dt.month
  data['Year'] = data['Date'].dt.year

  cols = list(data.columns)
  new_cols = [cols[0]] + cols[-3:] + cols[1:-3]
  data = data[new_cols]
  data = data.copy()
  data.drop('Date', axis=1, inplace=True)

  data = process_MealName_column(data)

  data['meal_time'] = data.MealName.apply(lambda x: 1 if 'breakfast' in x else 2 if 'lunch' in x else 3 if 'dinner' in x else 0)
  data['is_snack'] = data.MealName.apply(lambda x: 1 if 'snack' in x else 0)
  data['is_drinks'] = data.MealName.apply(lambda x: 1 if 'drinks' in x else 0)
  data['is_supplement'] = data.MealName.apply(lambda x: 1 if 'supplement' in x else 0)
  data['is_workout'] = data.MealName.apply(lambda x: 1 if 'workout' in x else 0)

  data.drop('MealName', axis=1, inplace=True)

  cols_ = data.columns
  new_cols = cols_[:5].tolist() + cols_[-5:].tolist() + cols_[5:-5].tolist()
  data = data[new_cols]


  data = convert_objects_into_numeric(data)


  return data

# **Fill missing values**

In [ ]:
def fill_missing_values(data):
  data.fillna(0, inplace=True)
  return data

# **Drop all columns which includes constant values more than 75%**

In [ ]:
def drop_unnecessary_columns(data):

    columns_to_drop = pd.read_csv('/content/drive/MyDrive/Final Project/columns_to_drop.csv').to_numpy().flatten().tolist()

    columns_to_drop = []
    for column in data.columns[10:-2]:
        if (data[column].value_counts().iloc[0] / data.shape[0]) > 0.75:
            columns_to_drop.append(column)
    data.drop(columns=columns_to_drop, axis=1, inplace=True)
    #pd.DataFrame(columns_to_drop).to_csv('/content/drive/MyDrive/Final Project/columns_to_drop.csv', index=False)
    return data

# **Round data**

In [ ]:
def round_data(data):
  round_columns = round_columns = list(data.columns[1: 4]) + list(data.columns[10:-2]) + [data.columns[-1]]
  data[round_columns] = data[round_columns].round(5)
  return data

# **Create new column "success_index" according to how customers reached their goal values in each category**

In [ ]:
def assign_success_index(data):
    total_columns = data.columns[27:28+16]
    goal_columns = data.columns[28+16:28+17+16]

    data[total_columns] = data[total_columns].fillna(0)
    data[goal_columns] = data[goal_columns].fillna(0)

    success_index_df = pd.DataFrame(index=data.index)

    for total_col, goal_col in zip(total_columns, goal_columns):
        success_index_df[f'{total_col}_success_index'] = abs(data[total_col] - data[goal_col])

    data['success_index'] = success_index_df.sum(axis=1)

    return data

# **Handle outliers**

In [ ]:
def process_outliers_iteratively(df, column):
    while True:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_whisker = Q1 - 1.5 * IQR
        upper_whisker = Q3 + 1.5 * IQR

        lower_35 = df[column].quantile(0.175)
        upper_35 = df[column].quantile(0.825)

        df.loc[(df[column] < lower_35) & (df[column] >= lower_whisker), column] = lower_whisker
        df.loc[(df[column] > upper_35) & (df[column] <= upper_whisker), column] = upper_whisker

        remaining_outliers = df[(df[column] < lower_whisker) | (df[column] > upper_whisker)]

        if remaining_outliers.empty:
            break

        df.loc[remaining_outliers.index, column] = df.loc[remaining_outliers.index, column].apply(
            lambda x: lower_whisker if x < lower_whisker else upper_whisker)

    return df[column]

In [ ]:
def handle_outliers(data):
    for i in data.columns[10:-2]:
        data[i] = process_outliers_iteratively(data, i)
    return data

# **All function execution**

In [ ]:
def all_functions(data):
    train_ = preprocess_data(data)
    del data
    filled = fill_missing_values(train_)
    del train_
    #get_min_max(train_).to_csv('/content/drive/MyDrive/Final Project/min_max.csv', index=False)
    scaled_data = scale_data(filled)
    del filled
    train = assign_success_index(scaled_data)
    # Removed deletion of scaled_data
    scaled_data_abs = get_abs(scaled_data) # Now you can use scaled_data here
    del scaled_data # Now you can safely delete scaled_data
    train = drop_unnecessary_columns(scaled_data_abs)
    del scaled_data_abs
    train.sort_values(by='Id', inplace=True, ascending=True)
    train_rounded = round_data(train)
    del train
    train = handle_outliers(train_rounded)
    del train_rounded
    train.to_csv('/content/drive/MyDrive/Final Project/pre_processed_test.csv')

In [ ]:
pd.read_csv('/content/drive/MyDrive/Final Project/min_max.csv')

,Column,Max,Min
0,Day,31.0,1.0
1,Month,12.0,1.0
2,Year,2015.0,2014.0
3,Sequence,6.0,1.0
4,meal_time,3.0,0.0
5,is_snack,1.0,0.0
6,is_drinks,1.0,0.0
7,is_supplement,1.0,0.0
8,is_workout,1.0,0.0
9,Calcium_value,833.0,0.0


# **For test data**

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Final Project/test.csv', low_memory=False, parse_dates=['Date'])

In [ ]:
all_functions(test)

Column Trn Fat_goal has min_val == max_val == 0.0. Filling this column with zeros.
